# Rasterization from Scratch: Drawing with Classic Algorithms

## Prepare drawing environment


In [1]:
from PIL import Image


def create_image(
    width: int = 200,
    height: int = 200,
    color:  float | tuple[float, ...] | str = 'white',
):
    """
    Create an image with the specified width, height, and color.

    Args:
        width (int): Width of the image.
        height (int): Height of the image.
        color (float | tuple[float, ...] | str): Color of the image. Can be a single float,
            a tuple of floats, or a string representing a color.

    Returns:
        Image: A PIL Image object with the specified dimensions and color.
    """

    return Image.new('RGB', (width, height), color)